In [1]:
#pip install yfinance

In [2]:
import yfinance as yf

def get_stock_data(ticker):
    # Fetch data for the specified ticker
    stock_data = yf.download(ticker)
    
    return stock_data

In [3]:
import pandas as pd  
     
# making data frame  
df = pd.read_csv("stockerbot-export.csv")  
   
df = df.iloc[:, :8]
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%a %b %d %H:%M:%S +0000 %Y', errors='coerce')
df = df.dropna(subset=['timestamp'])

# Access date components for each timestamp
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['year'] = df['timestamp'].dt.year

df['Price Day Before Tweet'] = pd.NA
df['Price Day of Tweet'] = pd.NA
df['Price Day After Tweet'] = pd.NA

In [4]:
df.head(5)

,id,text,timestamp,source,symbols,company_names,url,verified,month,day,year,Price Day Before Tweet,Price Day of Tweet,Price Day After Tweet
0,1.0197E+18,VIDEO: “I was in my office. I was minding my o...,2018-07-18 21:33:26,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,TRUE,7,18,2018,<NA>,<NA>,<NA>
1,1.01971E+18,The price of lumber $LB_F is down 22% since hi...,2018-07-18 22:22:47,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,TRUE,7,18,2018,<NA>,<NA>,<NA>
2,1.01971E+18,Who says the American Dream is dead? https://t...,2018-07-18 22:32:01,TheStreet,AIG,American,https://buff.ly/2L3kmc4,TRUE,7,18,2018,<NA>,<NA>,<NA>
3,1.01972E+18,Barry Silbert is extremely optimistic on bitco...,2018-07-18 22:52:52,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,TRUE,7,18,2018,<NA>,<NA>,<NA>
4,1.01972E+18,How satellites avoid attacks and space junk wh...,2018-07-18 23:00:01,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,TRUE,7,18,2018,<NA>,<NA>,<NA>


In [5]:
pd.set_option('display.max_rows', None)
ticker_lst = df['symbols'].unique().tolist()
print(len(ticker_lst))

453


In [6]:
stock = yf.Ticker('M')
stock_df = stock.history(period="10y")

In [7]:
def create_df(ticker_lst):
    ticker_dfs = {}
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        stock_d = stock.history(period="10y")
        stock_df = stock_d.loc['2018-01-01':'2018-12-31']
        
        ticker_dfs[ticker] = stock_df
        ticker_dfs[ticker].reset_index(inplace=True)
        #ticker_dfs[ticker]['Date'] = pd.to_datetime(ticker_dfs[ticker]['Date'], format='%a %b %d %H:%M:%S +0000 %Y')
    return ticker_dfs

In [ ]:
hi = create_df(ticker_lst)

FB-GOOGL-GOOG: No data found, symbol may be delisted
HRS: No data found, symbol may be delisted
INTC-USB: No data found, symbol may be delisted
AMZN-GPS: No data found, symbol may be delisted
TICKER: No data found, symbol may be delisted
CBS: No data found, symbol may be delisted
TWTR: No data found, symbol may be delisted
FB: No data found, symbol may be delisted
INFO: No data found, symbol may be delisted
JEC: No data found, symbol may be delisted
FNSR: No data found, symbol may be delisted
RE: No data found, symbol may be delisted
TMK: No data found, symbol may be delisted
NUAN: No data found, symbol may be delisted
TRQ: No data found, symbol may be delisted
DCIX: No data found, symbol may be delisted
FBHS: No data found, symbol may be delisted
PAH: No price data found, symbol may be delisted (period=10y)
WYN: No price data found, symbol may be delisted (period=10y)
LUK: No price data found, symbol may be delisted (period=10y)
FMSA: No price data found, symbol may be delisted (perio

In [ ]:
print(hi.shape)

In [ ]:
#print(hi['AIG'])

In [ ]:
def add_price_data(stock_dict, tweet_df, ticker_lst):
    for ticker in ticker_lst:
        stock_df = stock_dict[ticker]
        for index, row in tweet_df.iterrows():
            # Check if the stock ticker is 'AAPL'
            if ticker == row['symbols']:
                tweet_date = row['timestamp']
                # Append the price data to the dataset
                day_before = tweet_date - pd.Timedelta(days=1)
                day_after = tweet_date + pd.Timedelta(days=1)
                

                # Extract the date string
                date_str = str(day_before.date())
                
                # Filter the DataFrame based on the date string
                filtered_df = stock_df.loc[stock_df['Date'] == date_str]
                
                if filtered_df.empty:
                    price_day_before = "N/A"
                
                else:
                    # Select the 'Close' column from the filtered DataFrame
                    close_series = filtered_df['Close']

                    # Convert the Series to a numpy array
                    close_array = close_series.values

                    # Access the first element of the numpy array
                    price_day_before = close_array[0]
                
                #MOST RECENT DAY AFTER
                tweet_df.at[index, 'Price Day Before Tweet'] = price_day_before
                #tweet_df.at[index, 'Price Day of Tweet'] = price_day_of
                #tweet_df.at[index, 'Price Day After Tweet'] = price_day_after
            
    return tweet_df

In [ ]:
result_df = add_price_data(hi, df, ticker_lst)

In [ ]:
result_df.head(20)

In [ ]:
result_df[result_df['symbols'] == 'UDR']

In [ ]:
hi['UDR']